# GPT-4 API: Extract Structured Data from PDFs

Before starting, create a `.env` file in the root directory of the project and add the following environment variables:

```bash
OPENAI_API_KEY=your_openai_api_key
```

## Step 1: Create a new Assistant with File Search Enabled

In [ ]:
!pip -q install python-dotenv

In [ ]:
# load OPENAI_API_KEY value from .env file
from dotenv import load_dotenv

load_dotenv()

In [67]:
from openai import OpenAI

client = OpenAI()

assistant_name = "Paper Data Extractor"

# List available assistants and create a new one if "Paper Data Extractor" is not in the list
assistants = client.beta.assistants.list()
assistant_names = [assistant.name for assistant in assistants.data]

if assistant_name not in assistant_names:
    assistant = client.beta.assistants.create(
        name=assistant_name,
        model="gpt-4-turbo",
        tools=[{"type": "file_search"}],
    )
    print(f"New assistant created: {assistant.id}")
else:
    assistant = assistants.data[assistant_names.index(assistant_name)]
    print(f"Assistant '{assistant_name}' found with ID: {assistant.id}")

Assistant created: asst_WQsHaxQTTCkAPUbn6nXS1kbr


## Step 2: Read the PDF file paths

In [68]:
# Read all files in "current dir/pdfs" and save to file_paths 
import os
file_paths = []

for root, dirs, files in os.walk("pdfs"):
    for file in files:
        if file.endswith(".pdf"):
            file_paths.append(os.path.join(root, file))

file_paths

['pdfs/1-Tracking Real Time Layoffs with SEC Filings - A Preliminary Investigation.pdf',
 'pdfs/2-Overnight Post-Earnings Announcement Drift and SEC Form 8-K Disclosures.pdf',
 'pdfs/3-Forecasting Stock Excess Returns With SEC 8-K Filings.pdf']

## Step 3: Extract the relevant information from a PDF

In [75]:
# Upload a single PDF to the assistant
file_path = file_paths[0]
message_file = client.files.create(
  file=open(file_path, "rb"), 
  purpose="assistants"
)
print(f"Uploaded '{file_path}' to OpenAI")

Uploaded 'pdfs/1-Tracking Real Time Layoffs with SEC Filings - A Preliminary Investigation.pdf' to OpenAI


In [76]:
# Extract the structured data from the PDF
prompt = """Extract the following data from the provided paper: Title, the research questions, the types of data used for the study, the size of the data set, the history of the dataset (i.e. how many years does it cover), the source of the data, the methods used to answer the research questions, the various metrics used for measuring, and the outcomes the authors found. Return the extracted structured data as a JSON object. Only respond with the JSON object, and do not respond with anything else.

Return your response as a structured JSON object using the following format:
''' 
{
  "title_of_paper": "What is the title of the paper?", // string: the title of the paper
  "research_questions": ["What is the research question?"], // array of strings: the research questions. If there are multiple research questions, list them all as separate items in the array
  "data_types": ["What types of data were used?"], // array of strings: the types of data used for the study. If there are multiple types of data, list them all as separate items in the array
  "data_size": "What is the size of the dataset?", // string: the size of the data set, i.e. number of observations, samples, etc.
  "data_history": "How many years does the dataset cover?", // string: the history of the dataset
  "data_sources": ["What are the sources of the data?"], // array of string: the sources of the data. If there are multiple sources, list them all as separate items in the array
  "methods": ["What methods were used to answer the research questions?"], // array of string: the methods used to answer the research questions. If there are multiple methods, list them all as separate items in the array
  "metrics": ["What metrics were used for measuring?"], // array of string: the various metrics used for measuring. If there are multiple metrics, list them all as separate items in the array
  "outcomes": ["What outcomes did the authors find?"] // array of string: the outcomes the authors found. If there are multiple outcomes, list them all as separate items in the array
}
'''

Response:
"""

# Create a conversation thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
            "attachments": [
                {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
            ],
        }
    ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_IueNY5vx0uHzxkc89uVUCiS4'])


In [77]:
# Request the assistant to run the thread and create a response
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
# Remove annotations from response
for index, annotation in enumerate(message_content.annotations):
    message_content.value = message_content.value.replace(annotation.text, "")

print(message_content.value)

```json
{
  "title_of_paper": "Tracking Real Time Layoffs with SEC Filings: A Preliminary Investigation",
  "research_questions": [
    "How are layoff indicators from 8-K filings correlated with the business cycle?",
    "How does the industry composition differ in 8-K filings?",
    "Are 8-K filings preemptive in relation to WARN notices for layoffs?"
  ],
  "data_types": ["Company layoff announcements", "SEC filings data", "WARN notices"],
  "data_size": "Over 7,000 layoffs identified, with detailed specifics provided for those linked to 8-K filings.",
  "data_history": "The dataset covers historical layoffs including during the Great Recession and the Covid pandemic.",
  "data_sources": ["SEC filings", "WARN notices", "Compustat employment data"],
  "methods": [
    "Natural Language Processing with BERT and Llama 2 models.",
    "Regression analysis with VAR model specification.",
    "Link-analysis between 8-K filings and WARN notices"
  ],
  "metrics": [
    "Number of layoffs",

## Step 4: Convert GPT's output to JSON

In [78]:
import re
import json

# Remove starting "```json" and ending "```" values from GPTs response
json_string = (
    message_content.value.replace("```json\n", "").replace("```", "").replace("\n", "")
)
# Remove annotations from JSON string
json_string = re.sub(r"【.*】", "", json_string)
# Convert message_content.value to JSON
json_response = json.loads(json_string)

json_response

{'title_of_paper': 'Tracking Real Time Layoffs with SEC Filings: A Preliminary Investigation',
 'research_questions': ['How are layoff indicators from 8-K filings correlated with the business cycle?',
  'How does the industry composition differ in 8-K filings?',
  'Are 8-K filings preemptive in relation to WARN notices for layoffs?'],
 'data_types': ['Company layoff announcements',
  'SEC filings data',
  'WARN notices'],
 'data_size': 'Over 7,000 layoffs identified, with detailed specifics provided for those linked to 8-K filings.',
 'data_history': 'The dataset covers historical layoffs including during the Great Recession and the Covid pandemic.',
 'data_sources': ['SEC filings', 'WARN notices', 'Compustat employment data'],
 'methods': ['Natural Language Processing with BERT and Llama 2 models.',
  'Regression analysis with VAR model specification.',
  'Link-analysis between 8-K filings and WARN notices'],
 'metrics': ['Number of layoffs',
  'Number of companies affected',
  'Perce

In [44]:
!pip -q install pandas


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [79]:
# Convert the JSON response to a pandas DataFrame
import pandas as pd

json_response_df = pd.DataFrame([json_response])

json_response_df

,title_of_paper,research_questions,data_types,data_size,data_history,data_sources,methods,metrics,outcomes
0,Tracking Real Time Layoffs with SEC Filings: A...,[How are layoff indicators from 8-K filings co...,"[Company layoff announcements, SEC filings dat...","Over 7,000 layoffs identified, with detailed s...",The dataset covers historical layoffs includin...,"[SEC filings, WARN notices, Compustat employme...",[Natural Language Processing with BERT and Lla...,"[Number of layoffs, Number of companies affect...",[Confirmed predictive power of the 8-K filings...


## Step 5: Store extracted data to Airtable

- Create a new base and table in [Airtable](https://airtable.com/)
- Create a new automation using the "When webhook received" trigger and "Create record" action
- Map the JSON fields to the fields in the Airtable table inside the "Create record" tab. For example, map the `title_of_paper` field to the `Title` field in the Airtable table.

Alternative storage options: Python Pandas dataframe saved to local Parquet file, MongoDB, SQL databases (MySQL, PostgreSQL, etc), DynamoDB, Google Sheets, etc.

In [32]:
!pip -q install requests


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [74]:
import requests

# The webhook URL for the table in Airtable
webhook_url = "https://hooks.airtable.com/workflows/v1/genericWebhook/apphgC5p4Jgz2VV5w/wflPQAUElCuobq4wO/wtrRmZljovvAMXQYJ"

headers = {"Content-Type": "application/json"}

# Make the POST request to insert the data
response = requests.post(webhook_url, json=json_response, headers=headers)

# Check the response
if response.status_code == 200:
    print("Data inserted successfully!")
else:
    print(f"Failed to insert data: {response.text}")

Data inserted successfully!


Airtable table result after processing the first two PDFs:

![Airtable Table](assets/Airtable-table.svg)

## Step 6: Process all PDF files

In [51]:
# Exercise: Develop methods to extract structured data from multiple PDFs 
# and insert them into the Airtable table.
# Bonus: Use `pandarallel` to parallelize the extraction process and speed up the data extraction.